### Import Libs

In [1]:
import numpy as np
from scipy import optimize

### Create X and Y

In [2]:
X=np.asarray([[1, 1], [-1, 1], [-1, -1], [1, -1]])
Y=np.asarray([-1, 1, -1, 1])

### The Dual SVM Classifier

In [3]:
class DualSVM:    
    def __init__(self, kernel):
        self.kernel = kernel
        self.alpha = None
    
    def fit(self, X, Y):
        m = len(Y)
        
        # init alpha
        self.alpha = np.asarray([0.25, 0.25, 0.25, 0.25])
        
        def obj_func(alpha, epsilon_t, X, Y, maximize_sign):
            # this sum is w/o the log terms
            obj_sum_part1 = 0
            
            # breaking into smaller sums for readability
            sum1 = sum2 = sum3 = sum4a = sum4b = sum5 = 0

            # skipping first term (as per given obj func)
            # also alpha indexes reduced by 1 since passing alpha_2 to alpha_4
            # (since we have to calc alpha_1 from these)
            for i in range(1, m):
                sum1 += alpha[i-1] * (Y[i] * Y[0])
                sum2 += alpha[i-1]

                sum3 += alpha[i-1] * (Y[i] * Y[0])
                sum4a += alpha[i-1] * (Y[i] * Y[0])
                #sum4b += alpha[i-1] * Y[i] * (np.dot(X[i], X[0]))
                sum4b += alpha[i-1] * Y[i] * self.kernel(X[i], X[0])
                
                for j in range(1, m):
                    #sum5 += alpha[i-1] * Y[i] * (np.dot(X[i], X[j])) * Y[j] * alpha[j-1]
                    sum5 += alpha[i-1] * Y[i] * self.kernel(X[i], X[j]) * Y[j] * alpha[j-1]

            sum1 = -sum1
            #sum3 = pow(-sum3, 2) * np.dot(X[0], X[0])
            sum3 = pow(-sum3, 2) * self.kernel(X[0], X[0])
            sum4a = -sum4a

            obj_sum_part1 = sum1 + sum2 - 0.5 * (sum3 + 2*(sum4a * Y[0] * sum4b) + sum5)
            
            # ===== 1st part done =====
            
            # this sum is the log terms
            obj_sum_part2 = 0
            sum6 = sum7 = 0
            
            for i in range(1, m):
                # print(sum(alpha))
                sum6 += np.log(alpha[i-1])
                sum7 += alpha[i-1] * (Y[i] * Y[0])
                
            sum7 = np.log(-sum7)
            
            # ===== 2nd part done =====
            
            obj_sum_part2 = epsilon_t * (sum6 + sum7)
            
            return maximize_sign * (obj_sum_part1 + obj_sum_part2)
            
        # using this contraint to keep the first log term positive
        def constraint1(alph):
            return sum(alph) - 1e-8        
        
        cons1 = {'type': 'ineq', 'fun': constraint1}
        
        # tunable hyper parameters
        # t = time instances
        t = 1000
        # epsilon tends to 0 w.r.t. to time 't'
        epsilon_t_list = np.linspace(start=1e-2, stop=1e-8, num=t, endpoint=True)
        
        
        for e_t in epsilon_t_list:            
            # we are optimizing over alphas except the first one
            # args has -1 in the end since we want to maximize
            optimal_soln = optimize.minimize(fun = obj_func, 
                                             x0 = self.alpha[1:], 
                                             args = (e_t, X, Y, -1), 
                                             constraints = cons1, 
                                             method = 'SLSQP')
            
            # update alphas as given by the solver
            self.alpha[1:] = optimal_soln.x
            
            # calculate alpha_1 using given formula
            self.alpha[0] = -sum([self.alpha[k] * (Y[k] * Y[0]) for k in range(1,m)])
        
        # to calc. bias (not asked in question though)
        postive_alpha_index = self.alpha > 1e-8
        self.support_vector_X = X[postive_alpha_index]
        self.support_vector_Y = Y[postive_alpha_index]
        

        # using equation 33 of Lecture notes "Worked SVMs ..."
        self.b = self.support_vector_Y[0] - sum([self.alpha[j] * Y[j] * \
                                              self.kernel(X[j], self.support_vector_X[0]) for j in range(m)])
            
          
        # ==================================== END OF FIT ====================================
    
        
        
    
    # using equation 34 of Lecture notes "Worked SVMs ..."
    def predict(self, X):
        m = np.shape(X)[0]
        return np.sign(sum([self.alpha[i] * self.support_vector_Y[i] * self.kernel(self.support_vector_X[i], X) \
                            + self.b for i in range(m)]))

### The Kernel Function

In [4]:
def polyKernel(x, y):
    return pow(1 + np.dot(x, y), 2)

In [5]:
# Create model
svm_model = DualSVM(kernel=polyKernel)

# Train model
svm_model.fit(X, Y)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in log
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in log


In [6]:
# alpha values
svm_model.alpha

array([0.12499954, 0.12503677, 0.125074  , 0.12503677])

In [7]:
# bias value
svm_model.b

-3.709480115521302e-06

In [8]:
svm_model.support_vector_X

array([[ 1,  1],
       [-1,  1],
       [-1, -1],
       [ 1, -1]])

In [9]:
svm_model.support_vector_Y

array([-1,  1, -1,  1])

In [10]:
# bias value
svm_model.b

-3.709480115521302e-06

In [10]:
print(svm_model.predict([1, 1]))
print(svm_model.predict([-1, 1]))
print(svm_model.predict([-1, -1]))
print(svm_model.predict([1, -1]))

-1.0
1.0
1.0
1.0
